In [1]:
import os
import pandas as pd

from algorithms.PatternExplorer import transaction_collector
from entity.LightNode import LightNode
from utils.DataLoader import DataLoader
from utils.Settings import Setting
from utils.ProjectPath import ProjectPath
from entity import LightCluster, LightNode

path = ProjectPath()
setting = Setting()
dataloader = DataLoader()

cluster_path = path.univ2_cluster_path
cluster_file_template = "cluster_{}.csv"
suspicious_file_template = "cluster_{}_suspicious_nodes.csv"

ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
LOAD END NODES
LOAD CREATION INFO


/home/dustin/Projects/SerialScammer/main/utils/DataLoader.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens["token0"] = tokens["token0"].str.lower()
/home/dustin/Projects/SerialScammer/main/utils/DataLoader.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens["token1"] = tokens["token1"].str.lower()


LOAD RUG PULL INFO


In [2]:
def count_hops_from_source(source_address, paths: list):
    source_address = source_address.lower()
    if not source_address in paths:
        return 0
    return len(paths[paths.index(source_address):])


def get_nodes_by_labels(root_label, target_labels, cluster: LightCluster):
    root_label_nodes = []
    target_label_nodes = dict()
    for node in cluster.nodes.values():
        if root_label in node.labels and not any(label in LightNode.LightNodeLabel.SKIP_LABELS for label in node.labels):
            root_label_nodes.append(node)
        for l in node.labels:
            if l in target_labels:
                nodes = []
                if l in target_label_nodes.keys():
                    nodes = target_label_nodes[l]
                nodes.append(node)
                target_label_nodes[l] = nodes
    return root_label_nodes, target_label_nodes


def find_impacts(root_node, target_nodes):
    satisfied_nodes = []
    hops = []
    for node in target_nodes:
        if root_node.address in node.path:
            satisfied_nodes.append(node)
            hops.append(count_hops_from_source(root_node.address, node.path))
    return satisfied_nodes, hops


def impact_analysis(root_label, target_labels, cluster: LightCluster):
    root_label_nodes, target_label_nodes = get_nodes_by_labels(root_label, target_labels, cluster)
    results = []
    for root in root_label_nodes:
        record = LightNode.LightNode.to_sort_dict(root)
        impacts = []
        for tl in target_label_nodes.keys():
            tgn = target_label_nodes.get(tl)
            satisfied_nodes, hops = find_impacts(root, tgn)
            impacts.append({"type": tl, "count": len(satisfied_nodes), "avg_distance": sum(hops) / len(hops) if len(hops) > 0 else 0})
        record["impacts"] = impacts
        results.append(record)
    return results




In [3]:
group_id = 3
cluster = LightCluster.LightCluster(group_id)
cluster.load(cluster_path)

In [4]:
results = impact_analysis(LightNode.LightNodeLabel.BIG_CONNECTOR, [LightNode.LightNodeLabel.BIG_CONNECTOR, LightNode.LightNodeLabel.SCAMMER, LightNode.LightNodeLabel.BIG_WASHTRADER], cluster)
display(results)

[]

In [33]:
df = pd.read_csv(os.path.join(path.panv2_processed_path, "1_pair_pool_labels.csv"))
display(df.head())

,pool,creator,is_rp,token0,token1,scam_token
0,0x804678fa97d91B974ec2af3c843270886528a9E6,0xfca08fd2057a995cd270f22076c902b5cd2b4237,0,0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82,0xe9e7cea3dedca5984780bafc599bd69add087d56,NaN
1,0x0eD7e52944161450477ee417DE9Cd3a859b14fD0,0x36469e4e6ec2fb3e06db1d747f3fc1f1fc762f02,0,0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,NaN
2,0x58F876857a02D6762E0101bb5C46A8c1ED44Dc16,0x36469e4e6ec2fb3e06db1d747f3fc1f1fc762f02,0,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe9e7cea3dedca5984780bafc599bd69add087d56,NaN
3,0x28415ff2C35b65B9E5c7de82126b4015ab9d031F,0x36469e4e6ec2fb3e06db1d747f3fc1f1fc762f02,0,0x3ee2200efb3400fabb9aacf31297cbdd1d435d47,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,NaN
4,0x168B273278F3A8d302De5E879aA30690B7E6c28f,0x36469e4e6ec2fb3e06db1d747f3fc1f1fc762f02,0,0xad6caeb32cd2c308980a548bd0bc5aa4306c6c18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,NaN


In [34]:
rp = df[df["is_rp"] == 1]
rp

,pool,creator,is_rp,token0,token1,scam_token
116,0x45c0a88988f33123794DAb66c232c00e7C1DdCd5,0xecaace2c92afe113057c1bf66c78ccd08e20d249,1,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xef54802c87bb66bc61f069e3e9f5953e322cff58,0xef54802c87bb66bc61f069e3e9f5953e322cff58
119,0xC403605005e2C878914ee9FEAD840490a961782b,0x7fb0792ad88320711270f219510bc3be928293c2,1,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xd1c24bcabab5f01bcba2b44b6097ba506be67d6d,0xd1c24bcabab5f01bcba2b44b6097ba506be67d6d
143,0xaaBd4586Ed416E14EBe6321523883EC36E387a47,0xecaace2c92afe113057c1bf66c78ccd08e20d249,1,0x3a5a4d66680a165863a9295f620ca17e17e8f0c1,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x3a5a4d66680a165863a9295f620ca17e17e8f0c1
175,0x00A02E8AE70e1EE6b95900c162084111e54C8306,0x95dff9d572a5503885b8f338a3fcc7fce1049081,1,0x0f3a658a5d78c684c5d6f4448eb12db206af3ef0,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x0f3a658a5d78c684c5d6f4448eb12db206af3ef0
188,0xaaE089869A32fD6826CeA32D416FdDF79B1F8644,0xd5bb7f22696b759d28ce767d0249a98ceaf29474,1,0x00f39213a7e6bd2badaada4c9e40bb4775e7ab88,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x00f39213a7e6bd2badaada4c9e40bb4775e7ab88
...,...,...,...,...,...,...
1609080,0x8f06f166C3713f6F39acF6c6Cb596C4D98F6ab61,0xe582fd2a5101de759ba5879c181fa6ed12c1a315,1,0x037776e89b9cd1dcdbe0213abd7634dbaadcec4c,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x037776e89b9cd1dcdbe0213abd7634dbaadcec4c
1609095,0x3f3033f63d8985833be4492AeA07bA784C27AD50,0x1b9a61d35f6b0422185bc57535550156ae801396,1,0xaa11b45fae62ee62782d3d4275a8fe23e9d995ef,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xaa11b45fae62ee62782d3d4275a8fe23e9d995ef
1609273,0xd76F31Ef55BedEE19d6649f444100D243a28B8b7,0xeb601b9ddb3d15146fba090d39ec202a4fb650ad,1,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xeef5a93fc343bc031ac756f3226b6e9aa0b37af0,0xeef5a93fc343bc031ac756f3226b6e9aa0b37af0
1609294,0x91b6aF946373E7c43DB836752cE1a55eb9F1Ad22,0x249ae5bd8bf15ebb00a642044dc26d4a4868efa5,1,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xffbbd44efe59e80c980fc94d06a7fc449208ce3c,0xffbbd44efe59e80c980fc94d06a7fc449208ce3c


In [7]:
pcs = pd.read_csv(os.path.join(path.panv2_processed_path, "1_pair_scammers.csv"))
print("Pancake unique scammers", len(pcs["scammer"].unique()))
uns = pd.read_csv(os.path.join(path.univ2_processed_path, "1_pair_scammers.csv"))
print("Uniswap unique scammers", len(uns["scammer"].unique()))
print("Total unique scammers", len(uns["scammer"].unique())+len(uns["scammer"].unique()))

Pancake unique scammers 244874
Uniswap unique scammers 190958
Total unique scammers 381916


In [41]:
from tqdm import tqdm
from data_collection.AccountCollector import TransactionCollector 
uns = pd.read_csv(os.path.join(path.univ2_processed_path, "filtered_simple_rp_scammers.csv"))
print("Uniswap unique scammers", len(uns["scammer"].unique()))
collector = TransactionCollector()
scammers = uns["scammer"].unique()
scammer_pools = uns.groupby("scammer")["pool"].apply(list).to_dict()
ntxs = []
for scammer in tqdm(scammers):
    nor_txs, int_txs = collector.get_transactions(scammer)
    pools = scammer_pools[scammer]
    ntxs.append({"scammer":scammer, "num_normal_txs": len(nor_txs), "num_pools": len(pools), "pools": ";".join(pools)})
ntxs_df = pd.DataFrame(ntxs)
ntxs_df.to_csv("filtered_simple_scammer_txs_size.csv", index=False)
ntxs_df = pd.read_csv("filtered_simple_scammer_txs_size.csv")
display(ntxs_df.head())

Uniswap unique scammers 176751


100%|██████████| 176751/176751 [11:08<00:00, 264.27it/s]


,scammer,num_normal_txs,num_pools,pools
0,0xe2772341c3ca68f332ec8d68d13b5ebae3dd26d8,219,1,0x2F2399AfBcF979cA3D92a3158562C9A397b28D06
1,0x6f9ff1f95bde025e3d5e4633304fa8c29ea8acf8,14,1,0x0e2129c1bDbb0AfaA6Be1ed53650B267C0FB5556
2,0xc1ff218cb1355e6c7b25e8144252cc5ea3c08d9f,16,1,0x7f6fe5c9FF1F9D46B29B79537D9ce6337127DFE0
3,0xae9943ffb0ea5959ddbf7bbe93d246c861d4c939,11,1,0x083B344D961D4Ab3823ACceC60253ec6c4bdB511
4,0xcb114805b901f7a9c38d5675272ef26459a7d805,5114,1,0x441d3A8eA2076ad7326E91f52704CBE00C911F73


In [42]:
zero_txs_scammers = ntxs_df[ntxs_df["num_normal_txs"] == 0]
display(zero_txs_scammers)

,scammer,num_normal_txs,num_pools,pools


In [44]:
over_thoundsandscammers = ntxs_df[ntxs_df["num_normal_txs"] > 1000]
over_thoundsandscammers

,scammer,num_normal_txs,num_pools,pools
4,0xcb114805b901f7a9c38d5675272ef26459a7d805,5114,1,0x441d3A8eA2076ad7326E91f52704CBE00C911F73
8,0x1a87b5ccb1c8257815bd2b0a87c899fac59ca598,5799,1,0xd5662ABf455beB1d81Ba46d62a82cb3D03e471fC
10,0x21fb8e54a34f462c3642b5dc3e529206fdf7b324,1230,2,0x14583f97FD7097fd70Bc57c2a35A77EaC9D1b27c;0xC...
13,0x89e2aebc454556f60f2b6facd1746bcb58f0b4f3,2217,1,0x497f4633dDfAcfc481f8C8cfA39F49bcf45b17F8
16,0x5d63095cd6a33b626ba08d121c889bd987dbba1c,1937,4,0x3fF6d7ADD4508d258e192AADC9eD1371869a4aB9;0x9...
...,...,...,...,...
172676,0xcc9c0cc7ba8fe212126a96b7bbef8bc62c53a13f,3807,1,0xC81950F7555d596913d0aB999681b59B43BABc8b
173406,0x9f49eb2d0d0c3065357b2deb39c18aca1b4cc2b9,1777,4,0xb34CCBa001997C31f2d9f51e7fc408D0bEa779B3;0x2...
174684,0x566842a1eb3dfa231574146f86aa2718850da5e2,2501,1,0x870352716AfbDec3cE29450D57f06c3060947434
174849,0x33e6d5c35117e00d66a656fae0be45621a964df9,1557,2,0xa86af204A3892FB71D5f87AD7bE0fe77D8236406;0x4...


In [25]:
normal_txs, internal_txs = collector.get_transactions("0x7059010320d07a47d2ad54db88bb89ea644c3e6f")
print(normal_txs)

[]
